In [2]:
import pandas as pd
import numpy as np
import time
import nltk
nltk.download('punkt')
from transformers import *
import matplotlib.pyplot as plt
import torch
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ken19980727\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df_kai = pd.read_csv('bc7_task3_valid.input0.tsv',sep = '\t')
df_me = pd.read_csv('Non-Process_data/BioCreative_ValTask3.tsv',sep = '\t')
df_kai

,source_text,target_text,predict_text
0,I'm 37 weeks pregnant so I can do whatever the...,0,0
1,@RaynaJaye bahahahahahahahaha,0,0
2,@Mamas_RipDad thank yu 😘,0,0
3,Bruh do you wanna fight nah man it's problem,0,0
4,It has been a difficult transition .,0,0
...,...,...,...
37893,@doritmi @KathyMcGrath4 Got my Tdap at 37 week...,1,1
37894,"Oaklee's heartbeat was 142 today,and I have fi...",1,1
37895,Got my whooping cough vaccine yesterday and no...,1,1
37896,Got myself a new bff! I'm now sat on my bed on...,1,1


In [6]:
df_kai[df_kai['predict_text'] == 1][df_kai['target_text'] == 1]

<ipython-input-6-497e8adacf80>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_kai[df_kai['predict_text'] == 1][df_kai['target_text'] == 1]


,source_text,target_text,predict_text
37805,Just get on birth control and use two condoms....,1,1
37806,I have a possible infection from my Zofran pum...,1,1
37807,@bethanygiuffre it's time for the epidural!,1,1
37809,"*puts cocoa butter on, takes prenatal vitamin,...",1,1
37810,"Update: I got the epidural, back contractions ...",1,1
...,...,...,...
37893,@doritmi @KathyMcGrath4 Got my Tdap at 37 week...,1,1
37894,"Oaklee's heartbeat was 142 today,and I have fi...",1,1
37895,Got my whooping cough vaccine yesterday and no...,1,1
37896,Got myself a new bff! I'm now sat on my bed on...,1,1


In [7]:
senior = [i for i in df_me['text']]
me = [i for i in df_kai['source_text']]

for idx,(i,j) in enumerate(zip(senior,me)):
    if i != j:
        if i.strip() in me:
            pass
        else:
            print(idx)
            print(i)

### ========================================================================================

In [8]:
model_ty = 'bert-base-cased'
# model_ty = 'bert-base-multilingual-cased'
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained(model_ty, do_lower_case=False)

In [9]:
MODEL = torch.load('Model/BERT_case_BC.pt')
with open('Model/tag_map.json') as jsonfile:
    tag2idx = json.load(jsonfile)
MODEL.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [10]:
idx2tag = {tag2idx[t]:t for t in tag2idx}
idx2tag

{0: 'I-', 1: 'O', 2: 'B-', 3: 'PAD'}

In [11]:
def predict(model,sent,o_sent):
    torch_sent = torch.LongTensor(torch.tensor([sent])).cuda()
    with torch.no_grad():
        output = model(torch_sent)[0][0]
        out_tag = [idx2tag[o] for o in torch.max(output,1)[1].cpu().numpy()]
    pred = []
    for s,t in zip(o_sent,out_tag):
        if s == '<ETY>':
            pred.append(t)
    return pred

In [12]:
sentences_pred = []
tags_pred = []
def first_wordpicee(w_p):
    w_p[0] = '<ETY>'
    return w_p
    
for i in df_kai.index:
    tokens = nltk.word_tokenize(df_kai['source_text'][i])
    if df_kai['predict_text'][i] == 1:
        SENT = []
        O_SENT = []
        for token in tokens:
            O_SENT.extend(first_wordpicee(tokenizer.tokenize(token)))
            SENT.extend(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(token)))
        tag = predict(MODEL,SENT,O_SENT)
        tags_pred.append(tag)
    else:
        tags_pred.append(len(tokens)*['O'])
    sentences_pred.append(tokens)
# sentences

In [13]:
for sent,tag in zip(sentences_pred,tags_pred):
    if len(sent) != len(tag):
        print('asd')
        break

In [14]:
IOB2_me = pd.read_csv("IOB2_Data/BC_dev_IOB2_all.txt",sep = '\t')
IOB2_me = IOB2_me.fillna('NA')
type(IOB2_me['Sentence#'][0])

numpy.int64

In [15]:
IOB2_me

,Word,Tag,Length,Sentence#
0,I,O,1,0
1,'m,O,2,0
2,37,O,2,0
3,weeks,O,5,0
4,pregnant,O,8,0
...,...,...,...,...
548096,another,O,7,37897
548097,day,O,3,37897
548098,I,O,1,37897
548099,'m,O,2,37897


In [16]:
sentences_true = []
tags_true = []
s_num = 0
s_ = []
t_ = []
for i in IOB2_me.index:
    if s_num != IOB2_me['Sentence#'][i]:
        s_num = IOB2_me['Sentence#'][i]
        sentences_true.append(s_)
        tags_true.append(t_)
        s_ = []
        t_ = []
        s_.append(IOB2_me['Word'][i])
        t_.append(IOB2_me['Tag'][i])
    else:
        s_.append(IOB2_me['Word'][i])
        t_.append(IOB2_me['Tag'][i])

In [17]:
len(sentences_true)

37897

In [18]:
labels = []
predicts = []
## 還未增加NA轉nan的狀態
for idt,s_t in enumerate(sentences_true):
    if s_t in sentences_pred:
        for idp,s_p in enumerate(sentences_pred):
            if s_t == s_p:
                labels.append(tags_true[idt])
                predicts.append(tags_pred[idp])
                break

In [19]:
from seqeval.scheme import IOB2
from seqeval.metrics import classification_report,f1_score

In [20]:
print(classification_report(labels , predicts,scheme = IOB2))

              precision    recall  f1-score   support

           _       0.90      0.58      0.70        99

   micro avg       0.90      0.58      0.70        99
   macro avg       0.90      0.58      0.70        99
weighted avg       0.90      0.58      0.70        99



In [37]:
def replace_NA(x):
    if x == 'NA':
        return 'nan'
    else:
        return x
error_sent = [] 
for i,j in zip(sentences_true , sentences_pred):
    if i != j:
        if i in sentences_pred:
            pass
        else:
            if list(map(replace_NA , i)) in sentences_pred:
                print('asd')
            else:
                print(i)

asd
asd
['Ugh', '.', 'My', 'next', 'syringe', 'change', 'is', 'going', 'to', 'be', 'around', '4', 'am', '.', 'Whyyy', '.', '#', 'LifeWithA', 'Zofran', 'Pump']
['@', 'daysmadeofnow', 'can', 'you', 'do', 'the', 'ibuprofen', '/', 'acetaminophen', 'switching', '?']
['\U000feb15I', 'hate', 'the', 'sleepy', 'side', 'effect', 'when', 'your', 'eggo', 'is', 'preggo', 'or', 'maybe', 'it', 'these', 'vitamins', '\U000feb14never', 'wear', 'panties', 'ever', 'but', 'I', '...', 'http', ':', '//t.co/vQnof2HBx3']
asd
['Type', '1', 'diabetes', '&', 'amp', ';', 'pregnancy', '...', 'She', 'has', 'a', 'really', 'funky', 'placenta', 'that', 'requires', 'the', 'most', 'ridiculous', 'dose', 'of', 'insulin', '.', '’', 'http', ':', '//t.co/piwBDkYkvd']
['Holy', 'shit', 'I', 'need', 'my', 'freaking', 'anxiety', 'meds', '😩']


In [39]:
ert = 'Ugh'
# ert2 = 'Pump'
for idx,i in enumerate(sentences_pred):
    if ert in i:
        print(i,idx)

['The', 'November', 'phone', 'bill', 'is', 'gon', 'na', 'be', 'so', 'high', '!', 'Ugh', 'activation', 'fees', 'lol'] 961
['@', 'RetMSgt', 'I', 'used', 'to', 'have', 'this', 'pain', 'in', 'my', 'neck', 'a', 'few', 'times', 'a', 'year', '.', 'It', "'s", 'so', 'horrible', 'and', 'I', 'feel', 'for', 'anyone', 'who', 'has', 'to', 'go', 'through', 'it', '.', 'Ugh', '.'] 1113
['Waiting', 'waiting', '.', 'Ugh'] 2465
['If', 'My', 'Aunt', 'Do', "n't", 'Give', 'Me', 'The', 'Key', 'To', 'Get', 'In', 'The', 'House', '!', '!', '!', 'Ugh'] 2749
['Why', 'am', 'I', 'so', 'tired', '?', 'I', 'hardly', 'did', 'anything', 'today', '!', 'Ugh', '.'] 3778
['Is', 'my', 'brother', 'being', 'a', 'fckn', 'hoe', 'again', '?', 'Ugh'] 3787
['Ugh', 'this', 'weather', 'change', 'and', 'my', 'sinuses', 'and', 'allergies', 'are', 'not', 'getting', 'along😭😭😭'] 4118
['Ugh', 'so', 'miserable'] 5206
['@', 'CheyCornwell', 'The', 'Way', 'My', 'Body', 'Is', 'Set', 'Up', 'Right', 'Now', 'Im', 'Ready', 'For', 'Her', 'To', 'Be', 

In [46]:
sentences[6590] == error_sent[0]

False

In [149]:
len(dev_sentences)

18952

In [104]:
c

37664